
PONTIFICIA UNIVERSIDAD JAVERIANA

Autor: J. Gonzalez.

Fecha: 22 de julio 2024

Tema: Inrtoducccion al procesamiento de altos volumenes de datos


In [0]:
##Importar las bibliotecas necesarias.
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType

In [0]:
##DataBricks es un entorno que proporciona cómputo y almacenamiento.
##Community Edition permite acceder a parte de éstos recursos de forma gratuita.

## Se observa los datasets (conjunto de datos) disponibles en DataBricks.

display(dbutils.fs.ls("databricks-datasets/samples"))

path,name,size,modificationTime
dbfs:/databricks-datasets/samples/adam/,adam/,0,0
dbfs:/databricks-datasets/samples/data/,data/,0,0
dbfs:/databricks-datasets/samples/docs/,docs/,0,0
dbfs:/databricks-datasets/samples/lending_club/,lending_club/,0,0
dbfs:/databricks-datasets/samples/newsgroups/,newsgroups/,0,0
dbfs:/databricks-datasets/samples/people/,people/,0,0
dbfs:/databricks-datasets/samples/population-vs-price/,population-vs-price/,0,0


In [0]:
## como ejemplo se carga un fichero
## la carga se instancia en un objeto DATAFRAME de Spark
dfejemplo00 = spark.read.csv(
    "/databricks-datasets/samples/population-vs-price/data_geo.csv", header=True
)
## se presenta el dataframe
dfejemplo00.show(4)

+---------+----------+-------+----------+------------------------+-----------------------+
|2014 rank|      City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+----------+-------+----------+------------------------+-----------------------+
|      101|Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|    Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|Montgomery|Alabama|        AL|                  200481|                    129|
+---------+----------+-------+----------+------------------------+-----------------------+
only showing top 4 rows



In [0]:
## se imprimen los nombres de ls columnas.

dfejemplo00.columns

['2014 rank',
 'City',
 'State',
 'State Code',
 '2014 Population estimate',
 '2015 median sales price']

In [0]:
## se presenta un resumen estadístico

dfejemplo00.describe().show()

+-------+-----------------+-------+---------+----------+------------------------+-----------------------+
|summary|        2014 rank|   City|    State|State Code|2014 Population estimate|2015 median sales price|
+-------+-----------------+-------+---------+----------+------------------------+-----------------------+
|  count|              294|    294|      294|       294|                     293|                    109|
|   mean|            147.5|   NULL|     NULL|      NULL|      307284.89761092153|      211.2660550458717|
| stddev|85.01470461043782|   NULL|     NULL|      NULL|       603487.8272175139|       134.017245449277|
|    min|                1|Abilene|  Alabama|        AK|                  101408|                  105.2|
|    max|               99|Yonkers|Wisconsin|        WI|                  912791|                   95.4|
+-------+-----------------+-------+---------+----------+------------------------+-----------------------+



In [0]:
## comanda para presentar el esquema

dfejemplo00.printSchema()

root
 |-- 2014 rank: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- 2014 Population estimate: string (nullable = true)
 |-- 2015 median sales price: string (nullable = true)



In [0]:
## se recomienda cambiar los nombres de las columnas

dfejemplo01 = dfejemplo00.withColumnRenamed("2014 rank", "2014_rank")\
    .withColumnRenamed('State Code','State_Code')\
    .withColumnRenamed('2014 Population estimate','2014_Population_estimate')\
    .withColumnRenamed('2015 median sales price','2015_median_sales_price')

display(dfejemplo01)

## Observaciones con los tipos de datos:
  - Se observa que todos los datos son del tipo STRIN, incluso aquellos que deben ser del tipo numérico
  - Se debe ajustar los datos a sus tipos de datos correspondientes
  - Para lo anterior se usará el método **.withColumn()** (cambio de tipo manual)
    


In [0]:
#se hace el casteo o cambio de tipo de datos

dfejemplo02 = dfejemplo01.withColumn("2014_rank", col("2014_rank").cast(IntegerType()))\
    .withColumn("2014_Population_estimate", col("2014_Population_estimate").cast(IntegerType()))\
    .withColumn("2015_median_sales_price", col("2015_median_sales_price").cast(FloatType()))

dfejemplo02.printSchema()
    


root
 |-- 2014_rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- 2014_Population_estimate: integer (nullable = true)
 |-- 2015_median_sales_price: float (nullable = true)



In [0]:
## Se revisa de nuevo el estadístico

display(dfejemplo02.describe())

summary,2014_rank,City,State,State_Code,2014_Population_estimate,2015_median_sales_price
count,294,294,294,294,293,109
mean,147.5,null,null,null,307284.89761092153,211.26605476589378
stddev,85.01470461043782,null,null,null,603487.8272175139,134.01724508539056
min,1,Abilene,Alabama,AK,101408,78.6
max,294,Yonkers,Wisconsin,WI,8491079,900.0



### Vista de datos

El comando **createOrReplaceTempView** crea una vista temporal usando un SparkDataframe en la sesión **Spark**. Éste permite hacer consultas *sql* sobre el objeto.

In [0]:
## Se hace una vista temporal

dfejemplo02.createOrReplaceTempView("date_geo")


  - Con **%sql** se crea una instruccion "mágica" para hacer consultas usando sql, es importante recalcar que debe colocarse al principio del codigo, ya que si no se coloca, puede no leer la instruccion.
  - Tambien se puede buscar en la parte superior derecha de la celda, la opcion de lenguaje.
  - Se puede hacer consultas SQL sobre el objeto de vista temporal.


In [0]:
%sql

SELECT `State_Code`, `2015_median_sales_price` 
FROM date_geo
LIMIT 10

State_Code,2015_median_sales_price
AL,162.9
AL,157.7
AL,122.5
AL,129.0
AK,null
AZ,null
AZ,null
AZ,null
AZ,null
AZ,null
